## Segmenting and Clustering Neighborhoods in Toronto, Canada

#### By: Jessica Jacobson

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

In [29]:
#Import necessary libraries for notebook

!pip install beautifulsoup4

import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import requests #library to handle requests
from bs4 import BeautifulSoup # used to scrape data from Wikipedia page

print ("Libraries Imported Successfully")

Libraries Imported Successfully


In [30]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url)
soup = BeautifulSoup(source.content, 'html5lib') 
table=soup.find('table') 
print ("Data Downloaded Successfully!")

Data Downloaded Successfully!


### Now that the data has been scraped from the website, we must convert the table to a pandas dataframe.

#### First, we create an empty dataframe with our desired column names

In [31]:
#The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
column_names = ['PostalCode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)
df

,PostalCode,Borough,Neighborhood


#### Next, using a loop, we fill our dataframe with information from the website

In [32]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [33]:
#Print the first 5 rows of our dataframe
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Data Cleaning

In [36]:
# Remove rows where Borough is 'Not assigned'
df = df[df.Borough != 'Not assigned']

# Combine the neighbourhoods with same Postalcode
df = df.groupby(['PostalCode','Borough'], sort=False).agg(', '.join)
df.reset_index(inplace=True)

# For neighborhoods listed as "unassigned," replace with the name of the Borough
df['Neighborhood'] = np.where(df['Neighborhood'] == 'Not assigned',df['Borough'], df['Neighborhood'])

# Print first 10 rows of dataframe to view results
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [38]:
df.shape

(103, 3)

##### Our dataframe has 103 rows

## Add latitude and longitude coordinates for each zipcode in our dataframe

In [42]:
Geo_data=pd.read_csv('http://cocl.us/Geospatial_data')
Geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [43]:
# Rename columns in Geo_data dataframe and merge datasets based on PostalCode column
Geo_data.rename(columns={'Postal Code':'PostalCode'},inplace=True)
df2 = pd.merge(Geo_data, df, on='PostalCode')

#Print first 10 rows of new dataframe
df2.head(10)

,PostalCode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae
5,M1J,43.744734,-79.239476,Scarborough,Scarborough Village
6,M1K,43.727929,-79.262029,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,43.711112,-79.284577,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,43.716316,-79.239476,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,43.692657,-79.264848,Scarborough,"Birch Cliff, Cliffside West"
